# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [18]:
import numpy as np
!cat publications.tsv

pub_date	title	venue	excerpt	paper_url	slides_url	type
1/1/2022	Mental health and wellbeing during the COVID-19 period in Australia ANU Centre for Social Research and Methods			https://apo.org.au/node/318500		report
1/1/2021	The wellbeing of Australia�s children			https://assets-us-01.kc-usercontent.com/99f113b4-e5f7-00d2-23c0-c83ca2e4cfa2/e0b64280-dd99-4237-9c69-eaebc2ff3ce7/Australian-Childrens-Wellbeing-Index-Report.pdf		report
1/1/2022	Wellbeing outcomes in Australia as lockdowns ease and cases increase�August 2022			https://apo.org.au/node/319629		report
1/1/2022	The relationship between COVID-19 policies and subjective wellbeing�August 2022 ANU Centre for Social Research and Methods			https://scholar.google.com/scholar?cluster=2747851349458022468&hl=en&oi=scholarr		report
1/1/2022	Mental health and wellbeing during the COVID-19 period in Australia			https://csrm.cass.anu.edu.au/sites/default/files/docs/2022/7/Mental_health_and_wellbeing_during_the_COVID-19_period.pdf		report
1/1/

## Import pandas

We are using the very handy pandas library for dataframes.

In [19]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [20]:
publications = pd.read_excel("publications.xlsx", header=0)

def create_id(row):
    # Extract the last name of the first author
    first_author = row['authors'].split(';')[0].split(',')[0].strip()
    
    # Find the first word in the title that isn't "the" or "a"
    significant_word = next(word for word in row['title'].split() if word.lower() not in ['the', 'a'])
    
    # Combine with the year
    return f"{first_author}_{significant_word}_{str(row['pub_date'].date())}"

publications['url_slug'] = publications.apply(create_id, axis=1)
publications['id'] = publications.url_slug
publications['venue'] = [venue if not pd.isna(venue) else "" for venue in publications['venue']]

publications


,pub_date,title,venue,excerpt,paper_url,slides_url,type,authors,url_slug,id
0,2022-01-01,Mental health and wellbeing during the COVID-1...,,NaN,https://apo.org.au/node/318500,NaN,reports,"Biddle, Nicholas; Gray, Matthew; Rehill, Patri...",Biddle_Mental_2022-01-01,Biddle_Mental_2022-01-01
1,2021-01-01,The wellbeing of Australia’s children,,NaN,https://assets-us-01.kc-usercontent.com/99f113...,NaN,reports,"Noble, Kristy; Rehill, Patrick; Sollis, Kate; ...",Noble_wellbeing_2021-01-01,Noble_wellbeing_2021-01-01
2,2022-01-01,Wellbeing outcomes in Australia as lockdowns e...,,NaN,https://apo.org.au/node/319629,NaN,reports,"Biddle, Nicholas; Edwards, Ben; Gray, Matthew;...",Biddle_Wellbeing_2022-01-01,Biddle_Wellbeing_2022-01-01
3,2022-01-01,The relationship between COVID-19 policies and...,,NaN,https://scholar.google.com/scholar?cluster=274...,NaN,reports,"Biddle, Nicholas; Edwards, Ben; Rehill, Patrick;",Biddle_relationship_2022-01-01,Biddle_relationship_2022-01-01
4,2022-01-01,Mental health and wellbeing during the COVID-1...,,NaN,https://csrm.cass.anu.edu.au/sites/default/fil...,NaN,reports,"Biddle, Nicholas; Gray, Matthew; Rehill, Patri...",Biddle_Mental_2022-01-01,Biddle_Mental_2022-01-01
5,2020-01-01,Gender insights in the Solomon Islands: Findin...,,NaN,https://scholar.google.com/scholar?cluster=531...,NaN,reports,"Fisk, Kylie; McInerney, Carol; Rehill, Patrick...",Fisk_Gender_2020-01-01,Fisk_Gender_2020-01-01
6,2024-01-01,Policy learning for many outcomes of interest:...,Computational Economics,This explores the use of interpretable tree mo...,https://doi.org/10.1007/s10614-024-10722-1,NaN,manuscripts,"Rehill, Patrick; Biddle, Nicholas;",Rehill_Policy_2024-01-01,Rehill_Policy_2024-01-01
7,2021-01-01,Guidance note: Equality Insights and individua...,,NaN,https://scholar.google.com/scholar?cluster=539...,NaN,reports,"Crawford, Joanne; Fisk, Kylie; Pradela, Joanna...",Crawford_Guidance_2021-01-01,Crawford_Guidance_2021-01-01
8,2022-01-01,Variation in policy response to COVID-19 acros...,,NaN,https://www.bsg.ox.ac.uk/sites/default/files/2...,NaN,reports,"Edwards, Ben; Barnes, Roy; Rehill, Patrick; El...",Edwards_Variation_2022-01-01,Edwards_Variation_2022-01-01
9,2023-01-01,Fairness Implications of Heterogeneous Treatme...,arXiv preprint arXiv:2309.00805,NaN,https://arxiv.org/pdf/2309.00805,NaN,working_papers,"Rehill, Patrick; Biddle, Nicholas;",Rehill_Fairness_2023-01-01,Rehill_Fairness_2023-01-01


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [21]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [22]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.id) + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = str(item.pub_date)
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """\ncollection: publication"""
    
    md += """\ncategory: """ + item.type
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    # md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    # md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [23]:
!ls ../_publications/

Biddle_Mental_2022-01-01.md	   Rehill_Fairness_2023-01-01.md
Biddle_relationship_2022-01-01.md  Rehill_How_2024-01-01.md
Biddle_Wellbeing_2022-01-01.md	   Rehill_How_2025-12-02.md
Crawford_Guidance_2021-01-01.md    Rehill_Policy_2024-01-01.md
Edwards_Variation_2022-01-01.md    Rehill_Randomised_2025-01-04.md
Fisk_Gender_2020-01-01.md	   Rehill_Transparency_2024-01-01.md
Noble_wellbeing_2021-01-01.md	   Rehill_Understanding_2025-09-05.md
Rehill_Causal_2024-01-01.md	   Taylor_Process_2023-01-01.md
Rehill_Distilling_2024-01-01.md


In [24]:
!cat ../_publications/Rehill_Policy_2024-01-01.md

---
title: "Policy learning for many outcomes of interest: Combining optimal policy trees with multi-objective bayesian optimisation"

collection: publication
category: manuscripts
permalink: /publication/2024-01-01 00:00:00-Rehill_Policy_2024-01-01
excerpt: 'This explores the use of interpretable tree models with multi-objective Bayesian optimisation to explore policy eligibility trade-offs.'
date: 2024-01-01 00:00:00
venue: 'Computational Economics'
paperurl: 'https://doi.org/10.1007/s10614-024-10722-1'
---
This explores the use of interpretable tree models with multi-objective Bayesian optimisation to explore policy eligibility trade-offs.

[Download paper here](https://doi.org/10.1007/s10614-024-10722-1)
